In [1]:
from catboost import CatBoostClassifier
import os
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle
import datatable as dt
import pandas as pd

In [2]:
def create_model(params):
    model = CatBoostClassifier(**params)
    return model   

def get_params(random):
    np.random.seed(random)
    params = {'random_seed' : random,
            'use_best_model' : True,
            'od_type' : "Iter",
            'od_wait' :100,
            'silent' : True}
    return params     

In [47]:
class ansamble():
    
    def __init__(self, size = -1, random = 12345):
        self.size = size
        self.random = random
        self.ansamble = []
        if size == -1:
            holder = input('введи дерикторию с моделями')
            self.load(holder)
        else:
            for i in range(size):
                params = get_params(self.random + i)
                self.ansamble.append(create_model(params))
    
    def fit(self, X, y, holder, first = 0):
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size = .1)
        k = first
        
        if first > self.size:
            return 'обучение не возможно, начальное значение за пределами количества ансамблей модели'

        
        for i in tqdm(self.ansamble[(first):]):
            k += 1
            i.fit(X_train, y_train, eval_set = (X_test, y_test), plot = False)
            filename = holder + 'model' + str(k)
            pickle.dump(i, open(filename, 'wb'))
            
    def predict(self, test):
        result = []
        for i in self.ansamble:
            result.append(i.predict_proba(test)[:, 1])
        result = np.mean(result, axis = 0) 
        return result
    
    def load(self, holder, load_all = True):
        files = os.listdir(holder)
        self.ansamble = []
        k = 0
        for i in files:
            if 'model' in i:
                link = holder+i
                self.ansamble.append(pickle.load(open(link, 'rb')))
                
            k += 1
            if not load_all:
                if k == self.size:
                    break
        if load_all:
            self.size = len(self.ansamble)
                
    def up_fit(self, X, y, holder):
        files = os.listdir(holder)
        last_model = 0
        for i in files:
            if 'model' in i:
                number = int(i.replace('model', ''))
                if number > last_model:
                    last_model = number
        if last_model == self.size:
            return 'ансамбль обучен'
        return self.fit(X, y, holder, first = last_model)

* ansamble:
 1. если при создание не указать размер, то ансамбль подгрузит все модели из деректории 
 2. если при указать размер(n), то ансамбль создаст n не обученных моделей
* fit - обучает ансамбль: 
 1. поочередно обучает каждую модель ансамбля
 2. требует параметры:
     1. X - фичи для обучения
     2. y - таргетовое значение для обучения
     3. holder - целевая директория, куда будут сохраненны модели ансамбля
* up_fit - дообучает модель до указаного размера(задается при инициализации класса):
 1. не подгружает уже обученные можели в ансамбль для этого используй load
 2. требует параметры:
     1. X - фичи для обучения
     2. y - таргетовое значение для обучения
     3. holder - целевая директория, куда будут сохраненны модели ансамбля и спомощью которой будет определенно количество уже обученных моделей
* predict - предсказывает значения по тестовым фичам:
 1. требует параметры:
     1. test - фичи по которым нужно сделать предсказание
* load(self, holder, load_all = True):
 1. требует параметры:
     1. holder - целевая директория, откуда будут загруженны модели ансамбля
     2. load_all - по умолчанию True: показывает нужно ли загружать все модели или только n(размер ансамбля)